# 🔍 CrossFit RAG - Prueba del Retriever

Este notebook prueba que el retriever encuentra rutinas relevantes en ChromaDB.

> ⚠️ Asegurate de haber ejecutado `pipeline_ingesta.ipynb` completo antes de correr este notebook.

---
## ⚙️ Celda 1 - Setup
Importa el retriever y verifica que ChromaDB tiene datos cargados.

In [1]:
from rag.retriever import obtener_coleccion, recuperar_contexto

# Verifica conexión a ChromaDB
coleccion = obtener_coleccion()
print(f"✅ ChromaDB conectado")
print(f"📚 Semanas disponibles: {coleccion.count()}")

# Muestra las semanas cargadas
todos = coleccion.get(include=["metadatas"])
print()
for doc_id, meta in zip(todos["ids"], todos["metadatas"]):
    print(f"   → {doc_id} | inicio: {meta['fecha_inicio']} | olímpicos: {meta['movimientos_olimpicos'] or 'ninguno'}")

✅ ChromaDB conectado
📚 Semanas disponibles: 2

   → semana_2024_W06 | inicio: 2024-02-05 | olímpicos: clean, snatch
   → semana_2025_W07 | inicio: 2025-02-16 | olímpicos: clean, snatch


---
## 🔎 Celda 2 - Búsqueda general
Busca rutinas similares a un objetivo general.

In [2]:
contexto, rutinas = recuperar_contexto(
    objetivo="fuerza en tren inferior con sentadilla",
    n_resultados=3
)

print("Rutinas recuperadas:")
for r in rutinas:
    print(f"  → {r['semana_id']} | similitud: {r['similitud']} | patrones: {r['metadata']['patrones_fuerza']}")

🔍 Query de búsqueda: 'rutina de crossfit semanal enfocada en fuerza en tren inferior con sentadilla'
📚 Rutinas recuperadas: 2
   → semana_2025_W07 (similitud: 0.45)
   → semana_2024_W06 (similitud: 0.43)
Rutinas recuperadas:
  → semana_2025_W07 | similitud: 0.45 | patrones: OLY, empuje, sentadilla
  → semana_2024_W06 | similitud: 0.43 | patrones: OLY, empuje, sentadilla


---
## 🔎 Celda 3 - Búsqueda con parámetros específicos
Busca rutinas con movimientos olímpicos específicos y restricciones.

In [3]:
contexto, rutinas = recuperar_contexto(
    incluir_movimientos=["snatch"],
    evitar_movimientos=["clean"],
    intensidad="alta",
    n_resultados=3
)

print("Rutinas recuperadas:")
for r in rutinas:
    print(f"  → {r['semana_id']} | similitud: {r['similitud']} | olímpicos: {r['metadata']['movimientos_olimpicos']}")

🔍 Query de búsqueda: 'rutina de crossfit semanal con snatch sin repetir clean de intensidad alta'
📚 Rutinas recuperadas: 2
   → semana_2024_W06 (similitud: 0.44)
   → semana_2025_W07 (similitud: 0.439)
Rutinas recuperadas:
  → semana_2024_W06 | similitud: 0.44 | olímpicos: clean, snatch
  → semana_2025_W07 | similitud: 0.439 | olímpicos: clean, snatch


---
## 🔎 Celda 4 - Probá tu propia búsqueda
Modificá los parámetros para probar distintos escenarios.

In [4]:
# ✏️ Modificá estos parámetros para probar distintos escenarios
contexto, rutinas = recuperar_contexto(
    objetivo="trabajo de empuje y press",       # Cambiá esto
    evitar_movimientos=["sentadilla"],           # Cambiá esto
    incluir_movimientos=["press de banca"],      # Cambiá esto
    intensidad="media-alta",                     # Cambiá esto
    n_resultados=3
)

print("Rutinas recuperadas:")
for r in rutinas:
    print(f"  → {r['semana_id']} | similitud: {r['similitud']}")

# Muestra el contexto completo que recibirá el LLM generador
print("\n" + "="*50)
print("CONTEXTO QUE RECIBIRÁ EL GENERADOR:")
print("="*50)
print(contexto)

🔍 Query de búsqueda: 'rutina de crossfit semanal enfocada en trabajo de empuje y press con press de banca sin repetir sentadilla de intensidad media-alta'
📚 Rutinas recuperadas: 2
   → semana_2024_W06 (similitud: 0.32)
   → semana_2025_W07 (similitud: 0.302)
Rutinas recuperadas:
  → semana_2024_W06 | similitud: 0.32
  → semana_2025_W07 | similitud: 0.302

CONTEXTO QUE RECIBIRÁ EL GENERADOR:
RUTINAS DE REFERENCIA
Se recuperaron 2 rutinas similares para usar como base.

[Rutina 1 - semana_2024_W06 (similitud: 0.32)]
Olímpicos usados: clean, snatch
Patrones de fuerza: OLY, empuje, sentadilla
Grupos musculares: core, empuje, piernas

Semana: semana_2024_W06 | Inicio: 2024-02-05

LUNES - FUERZA
Core (3 rondas): Dead bug con disco 12, Plank superman 10/10, Hollow hold en barra 30"
FUERZA: Front Squat 5x4
Principiante: Goblet squat tempo 3–1–1
WOD AMRAP 14':
- Wall Ball 20
- Toes To Bar 10 (escala: Knee Raises 15)
- Burpee box jump over 8
Accesorios (3 rondas): Hip thrust 12, Step up DB 10/10

---
## ✅ Celda 5 - Verificación final
Confirma que el retriever está listo para conectarse con el generador.

In [5]:
# Prueba rápida con todos los parámetros posibles
contexto, rutinas = recuperar_contexto(
    objetivo="semana variada",
    n_resultados=3
)

# Verificaciones básicas
assert len(rutinas) > 0, "❌ El retriever no devolvió rutinas"
assert len(contexto) > 0, "❌ El contexto está vacío"
assert all("semana_id" in r for r in rutinas), "❌ Falta semana_id en alguna rutina"
assert all(0 <= r["similitud"] <= 1 for r in rutinas), "❌ Similitud fuera de rango"

print("✅ Todas las verificaciones pasaron")
print(f"   Rutinas recuperadas : {len(rutinas)}")
print(f"   Longitud del contexto: {len(contexto)} caracteres")
print(f"   Similitud promedio  : {round(sum(r['similitud'] for r in rutinas) / len(rutinas), 3)}")
print()
print("🎉 Retriever listo. El siguiente paso es generator.py")

🔍 Query de búsqueda: 'rutina de crossfit semanal enfocada en semana variada'
📚 Rutinas recuperadas: 2
   → semana_2025_W07 (similitud: 0.468)
   → semana_2024_W06 (similitud: 0.414)
✅ Todas las verificaciones pasaron
   Rutinas recuperadas : 2
   Longitud del contexto: 4575 caracteres
   Similitud promedio  : 0.441

🎉 Retriever listo. El siguiente paso es generator.py
